In [2]:
#This is my attempt to move my ET processing code to python so I can use the SSEBOP code

import datetime
import pprint

import ee
import pandas as pd

from IPython.display import Image
#to install openet.ssebop type "pip install openet-ssebop" in your cmd window
import openet.ssebop as model

#import geopandas as gpd #optional
#import folium #optional
ee.Initialize()#use_cloud_api=True)

In [3]:
#function to make code output into a table
def makeTable (info):
    #return pd.DataFrame.from_dict(info.items())
    #return pd.DataFrame(info.items(),columns=['ET','Value'])
    return pd.DataFrame.from_dict(info,orient='index',columns=["et"])

In [4]:
#Old functions from running the code a different way
def reduceFunc(image):
    median = image.reduceRegion(**{
    'reducer': ee.Reducer.median(),
    'scale':30,
    'geometry': study_area,
    'maxPixels': 1e11
    }).get('et')
    
    newFeature = ee.Feature(None,{'et':median}).copyProperties(image,['system:time_start'])
    
    return newFeature


def clipImageCol(image):
    image1 = image.clip(study_area)
    return image1;
  

In [5]:
monthly_df = {}

In [20]:
# this code chunk runs the monthly SSEBOP model for the specified years and feature. Output is a csv file written to Drive 
#with actual ET values.

#Need to also change file path in last line of this cell

  
###########################
#Inputs to change

study_crs = 'EPSG:32612'#'EPSG:6350' #'EPSG:5070'

#read in polygon
feat1 = ee.FeatureCollection('users/rlee/GovernmentCreekProjectAreas')
featPoly = feat1.filter(ee.Filter.eq("Project_ID",2552))

#choose years for analysis
my_list = list(range(2000,2020))

#################
#other inputs that you don't need to change
study_area = featPoly.geometry()
study_region = study_area


et_reference_source = 'IDAHO_EPSCOR/GRIDMET'
et_reference_band = 'etr'
et_reference_factor = 0.85
et_reference_resample = 'nearest'

cloud_cover = 70

interp_days = 32
# Interpolation method - currently only LINEAR is supported
interp_method = 'LINEAR'


###########################

#This is where the loop starts based on inputs. It takes 2+ hours to run for 2000-2020 for a given polygon. 


for item in my_list:
    yr = item 

    #if yr > 2013: #I took out L8 because I'm not sure that the model calibrates L7 and L8
    #2013-04-11T00:00:00 - 2020-10-02T00:00:00
    #    collections = ['LANDSAT/LC08/C01/T1_TOA'] 
    #elif yr <= 2013 and yr > 1999:
        #1999-01-01T00:00:00 - 2020-09-10T00:00:00
    #    collections = ['LANDSAT/LE07/C01/T1_TOA']#https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_TOA
    if yr > 1999:
        #1999-01-01T00:00:00 - 2020-09-10T00:00:00
        collections = ['LANDSAT/LE07/C01/T1_TOA']#https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_TOA
    else:
        #1984-01-01T00:00:00 - 2012-05-05T00:00:00
        collections = ['LANDSAT/LT05/C01/T1_TOA']

    for mo in range(1,13):
        start_date = f"{yr}-{mo:02}-01"
        if mo == 12:
            end_date = f"{yr+1}-01-01"
        else:
            end_date = f"{yr}-{mo+1:02}-01"
        
        model_obj = model.Collection(collections=collections, 
                                     et_reference_source=et_reference_source,
                                     et_reference_band=et_reference_band, 
                                     et_reference_factor=et_reference_factor,
                                     start_date=start_date,
                                     end_date=end_date, 
                                     geometry=study_area, #changed from test_point
                                     cloud_cover_max=70)

        monthly_coll = model_obj.interpolate(t_interval='monthly', variables=['et', 
                                                                              'et_reference', 
                                                                              'et_fraction'], 
                                             interp_method=interp_method, interp_days=interp_days,)

        
    #Fastest way to run et. This selects et. Could also select et_reference or et_fraction 
        monthly_df[start_date]=makeTable(monthly_coll.select(['et']).sum().reproject(crs=study_crs, scale=30).reduceRegion(reducer=ee.Reducer.median(),geometry= study_area,scale=30).getInfo())
        
        
    #These next three lines are the same as the one above, but broken into steps. Runs just barely slower on average
        #outimg = ee.Image(monthly_coll.select(['et']).sum()).reproject(crs=study_crs, scale=30).set('system:time_start',start_date)#.clip(study_area)
        #processOutImg = outimg.reduceRegion(reducer=ee.Reducer.median(),geometry= study_area,scale=30)#deleted crs
        #monthly_df[start_date]=makeTable(processOutImg.getInfo())
        
        
        print(monthly_df[start_date])
        print(start_date, end_date)
        

#check table
print(monthly_df)
print(pd.concat(monthly_df))


#Save et file 
pd.concat(monthly_df).to_csv(f"M:/Shared drives/UGS_Groundwater/Projects/WRI/Rebecca/PJNDVI/SSEBOP_ET/TestET2552L7Test.csv")

           et
et  13.432581
2000-01-01 2000-02-01
           et
et  31.430557
2000-02-01 2000-03-01
           et
et  69.414171
2000-03-01 2000-04-01
           et
et  52.716368
2000-04-01 2000-05-01
          et
et  89.78749
2000-05-01 2000-06-01
           et
et  87.318788
2000-06-01 2000-07-01
           et
et  54.782317
2000-07-01 2000-08-01
            et
et  108.787668
2000-08-01 2000-09-01
          et
et  62.12824
2000-09-01 2000-10-01
           et
et  31.151864
2000-10-01 2000-11-01
           et
et  43.308348
2000-11-01 2000-12-01
           et
et  24.135648
2000-12-01 2001-01-01
           et
et  36.422718
2001-01-01 2001-02-01
           et
et  47.386087
2001-02-01 2001-03-01
           et
et  85.830131
2001-03-01 2001-04-01
           et
et  76.752039
2001-04-01 2001-05-01
           et
et  92.765953
2001-05-01 2001-06-01
           et
et  71.208643
2001-06-01 2001-07-01
           et
et  78.235194
2001-07-01 2001-08-01
           et
et  40.789702
2001-08-01 2001-09-01
  

In [85]:
print(pd.concat(monthly_df))
#pd.concat(monthly_df).to_csv(f"M:/Shared drives/UGS_Groundwater/Projects/WRI/Rebecca/PJNDVI/SSEBOP_ET/TestET1628.csv")
#print(processImgCol.getInfo())

                       0
2000-01-01 et  18.631312
2000-02-01 et  35.847314
2000-03-01 et  49.790953
2000-04-01 et  43.272383
2000-05-01 et  51.766708
